## Load US CENSUS All-Sector Report data from 2017 datasets (see https://data.census.gov/cedsci/ and set your own parameters) for original sources

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

### We have local copies rooted in the S3_BUCKET : https://redhat-osc-physical-landing-647521352890.s3.amazonaws.com/US_CENSUS/US_Census-All-Sector-Report-2017/

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

## Based on OSC Ingest Tutorial by Erik Erlandson (GitHub: erikerlandson)

## python dependencies
The following are python packages commonly used for data ingest.
If your jupyter environment does not already have these,
you can copy and paste these into a live code cell to install them.

Run these in a notebook cell if you need to install onto your nb env
```
# 'capture' magic prevents long outputs from spamming your notebook
%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Standard python package for interacting with S3 buckets
%pip install boto3

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy[trino]

# Pandas and parquet file i/o
%pip install pandas pyarrow fastparquet

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools
```

## Environment variables and dot-env

The following cell looks for a "dot-env" file in some standard locations,
and loads its contents into `os.environ`.

In [1]:
import os
import pathlib
from dotenv import load_dotenv

# Load some standard environment variables from a dot-env file, if it exists.
# If no such file can be found, does not fail, and so allows these environment vars to
# be populated in some other way
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

## S3 and boto3

In [2]:
import boto3
s3_src = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
src_bucket = s3_src.Bucket(os.environ['S3_LANDING_BUCKET'])

## Connecting to Trino with sqlalchemy

In [3]:
import trino
from sqlalchemy.engine import create_engine

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ['TRINO_USER'],
    host = os.environ['TRINO_HOST'],
    port = os.environ['TRINO_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev',
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

## Load Data Dictionary for US Census 2017 All-sector report

In [4]:
import pandas as pd

data_dictionary = 'ECNCOMP2017.EC1700COMP_metadata_2021-11-21T141436.csv'
bObj = src_bucket.Object(f'US_CENSUS/US_Census-All-Sector-Report-2017/{data_dictionary}')
bObj.download_file(f'/tmp/ingest-{data_dictionary}')
dd_df = pd.read_csv(f'/tmp/ingest-{data_dictionary}', sep=',', header=None, engine='c')
dd_df.columns = ['name', 'desc'] 

os.unlink(f'/tmp/ingest-{data_dictionary}')
dd_df

,name,desc
0,GEO_ID,id
1,NAME,Geographic Area Name
2,GEO_ID_F,Geo Footnote
3,NAICS2012,2012 NAICS code
4,NAICS2012_F,2012 NAICS Footnote
5,NAICS2012_LABEL,Meaning of NAICS code
6,YEAR,Year
7,ESTAB,Number of establishments
8,RCPTOT,"Sales, value of shipments, or revenue ($1,000)"
9,PAYANN,"Annual payroll ($1,000)"


The US Census maintains a list of SIC codes, but it's very empty compared with the SEC version.  It does give 2-digit code definitions, however.  But they are not very complete, either.

In [5]:
survey_report = 'ECNCOMP2017.EC1700COMP_data_with_overlays_2021-11-21T141436.csv'
bObj = src_bucket.Object(f'US_CENSUS/US_Census-All-Sector-Report-2017/{survey_report}')
bObj.download_file(f'/tmp/ingest-{survey_report}')
df = pd.read_csv(f'/tmp/ingest-{survey_report}', sep=',', header=0, engine='c')
os.unlink(f'/tmp/ingest-{survey_report}')

df = df.drop(df.index[0])
df.reset_index (drop=True, inplace=True)
df

,GEO_ID,NAME,GEO_ID_F,NAICS2012,NAICS2012_F,NAICS2012_LABEL,YEAR,ESTAB,RCPTOT,PAYANN,EMP
0,0100000US,United States,NaN,21,NaN,"Mining, quarrying, and oil and gas extraction",2012,25417,529239818,59461950,848189
1,0100000US,United States,NaN,21,NaN,"Mining, quarrying, and oil and gas extraction",2017,25273,400465610,47500246,598620
2,0100000US,United States,NaN,211,NaN,Oil and gas extraction,2012,6735,310960365,15137960,176222
3,0100000US,United States,NaN,211,NaN,Oil and gas extraction,2017,6409,229734250,12624930,125129
4,0100000US,United States,NaN,2111,NaN,Oil and gas extraction,2012,6735,310960365,15137960,176222
...,...,...,...,...,...,...,...,...,...,...,...
3829,0100000US,United States,NaN,813920,NaN,Professional organizations,2017,6856,22275161,7025747,89047
3830,0100000US,United States,NaN,81399,NaN,"Other similar organizations (except business, ...",2012,20608,22116835,4948577,135074
3831,0100000US,United States,NaN,81399,NaN,"Other similar organizations (except business, ...",2017,20062,29076311,6075861,142619
3832,0100000US,United States,NaN,813990,NaN,"Other similar organizations (except business, ...",2012,20608,22116835,4948577,135074


## `osc-ingest-tools`

## Enforcing valid SQL column names

In [6]:
import osc_ingest_trino as osc
from io import BytesIO

osc.enforce_sql_column_names(df, inplace=True)
# Put year at the end to make for more friendly partitioning
year_index = df.columns.get_loc('year')
df = df[df.columns[:year_index].to_list()
                   + df.columns[year_index+1:].to_list()
                   + [df.columns[year_index]]]
df = df.convert_dtypes()

## Trino catalog, schema, table

In [7]:
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'sandbox'
census_table_prefix = 'census_'
ingest_table = 'all_sector_survey_2017'
dd_table = 'data_dictionary'
sic4_table = 'sic4'
# sic2_table = 'sic2'
sic_ranges_table = 'sic_ranges'

In [8]:
def create_census_table(tablename, df, partitioning=''):
    ingest_table = census_table_prefix+tablename
    columnschema = osc.create_table_schema_pairs(df, typemap={"datetime64[ns]":"timestamp(6)",
                                                              "datetime64[ns, UTC]":"timestamp(6)"})

    drop_table = engine.execute(f"drop table if exists {ingest_schema}.{ingest_table}")
    drop_table.fetchall()

    tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
{'partitioning = ' if partitioning else ''}{partitioning}{',' if partitioning else ''}
format = 'ORC'
)
"""
    print(tabledef)
    qres = engine.execute(tabledef)
    print(qres.fetchall())
    df.to_sql(ingest_table,
              con=engine, schema=ingest_schema, if_exists='append',
              index=False,
              method=osc.TrinoBatchInsert(batch_size = 10000, verbose = True))

In [9]:
create_census_table(ingest_table, df, "array['year']")


create table if not exists osc_datacommons_dev.sandbox.census_all_sector_survey_2017(
    geo_id varchar,
    name varchar,
    geo_id_f bigint,
    naics2012 varchar,
    naics2012_f varchar,
    naics2012_label varchar,
    estab varchar,
    rcptot varchar,
    payann varchar,
    emp varchar,
    year varchar
) with (
partitioning = array['year'],
format = 'ORC'
)

[(True,)]
constructed fully qualified table name as: "sandbox.census_all_sector_survey_2017"
inserting 3834 records
  ('0100000US', 'United States', NULL, '21', NULL, 'Mining, quarrying, and oil and gas extraction', '25417', '529239818', '59461950', '848189', '2012')
  ('0100000US', 'United States', NULL, '21', NULL, 'Mining, quarrying, and oil and gas extraction', '25273', '400465610', '47500246', '598620', '2017')
  ('0100000US', 'United States', NULL, '211', NULL, 'Oil and gas extraction', '6735', '310960365', '15137960', '176222', '2012')
  ...
  ('0100000US', 'United States', NULL, '813990', NULL, 'Other similar or

Load SIC4 codes from XML files (somewhat hidden) at SEC.GOV (https://www.sec.gov/xbrl/site/sic/2020/sic-lab-2020-01-31.xml)

In [10]:
# %pip install xmltodict

import xmltodict

with open('../data/external/sic-lab-2020-01-31.xml') as fd:
    doc = xmltodict.parse(fd.read())

sic4_xml = [x['#text'] for x in doc['link:linkbase']['link:labelLink']['link:label']]
title = sic4_xml[0]
df = pd.DataFrame(data=sic4_xml[1:], columns=['row'])
range_mask = df.row.str.contains(' to ')
df_ranges = df.loc[range_mask].copy()
df_defs = df.loc[~range_mask].copy()
df_ranges.info(verbose=True)
df_defs.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15 entries, 444 to 458
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   row     15 non-null     object
dtypes: object(1)
memory usage: 240.0+ bytes
<class 'pandas.core.frame.DataFrame'>
Int64Index: 444 entries, 0 to 443
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   row     444 non-null    object
dtypes: object(1)
memory usage: 6.9+ KB


In [11]:
def sic_range_tuple(x):
    l = x.split(' ', 3)
    r = l[3].rsplit(' ',1)
    return '-'.join([l[0], l[2]]), l[0], l[2], r[0]

temp = list(zip(*df_ranges['row'].map(sic_range_tuple)))
for i, c in enumerate(['range', 'l', 'r', 'range_desc']): df_ranges.loc[:, c] = temp[i]

df_ranges.l = df_ranges.l.astype('int32')
df_ranges.r = df_ranges.r.astype('int32')
df_ranges.drop(columns=['row'],inplace=True)
df_ranges.reset_index (drop=True, inplace=True)
df_ranges = df_ranges.convert_dtypes()

In [12]:
def sic_tuple(x):
    l = x.split(' ', 1)
    r = l[1].rsplit(' ',1)
    return l[0], r[0]

temp = list(zip(*df_defs['row'].map(sic_tuple)))
for i, c in enumerate(['sic', 'desc']): df_defs.loc[:, c] = temp[i]

df_defs.sic = df_defs.sic.astype('int32')
df_defs.drop(columns=['row'],inplace=True)
df_defs.reset_index (drop=True, inplace=True)
df_defs = df_defs.convert_dtypes()

In [13]:
create_census_table(sic4_table, df_defs)
create_census_table(sic_ranges_table, df_ranges)


create table if not exists osc_datacommons_dev.sandbox.census_sic4(
    sic integer,
    desc varchar
) with (

format = 'ORC'
)

[(True,)]
constructed fully qualified table name as: "sandbox.census_sic4"
inserting 444 records
  (100, 'Agricultural Production, Crops')
  (200, 'Agricultural Production, Livestock and Animal Specialties')
  (700, 'Agricultural Services')
  ...
  (9995, 'Nonoperating Establishments')
batch insert result: [(444,)]

create table if not exists osc_datacommons_dev.sandbox.census_sic_ranges(
    range varchar,
    l integer,
    r integer,
    range_desc varchar
) with (

format = 'ORC'
)

[(True,)]
constructed fully qualified table name as: "sandbox.census_sic_ranges"
inserting 15 records
  ('0000-0999', 0, 999, 'Agriculture, Forestry, and Fishing')
  ('5000-5199', 5000, 5199, 'Wholesale Trade')
  ('5200-5999', 5200, 5999, 'Retail Trade')
  ...
  ('1500-1799', 1500, 1799, 'Construction Industries')
batch insert result: [(15,)]


## SQL queries on our new table

In [14]:
sql = f"""
select * from {ingest_schema}.{census_table_prefix}{ingest_table}
"""
pd.read_sql(sql, engine).convert_dtypes()

,geo_id,name,geo_id_f,naics2012,naics2012_f,naics2012_label,estab,rcptot,payann,emp,year
0,0100000US,United States,None,21,<NA>,"Mining, quarrying, and oil and gas extraction",25273,400465610,47500246,598620,2017
1,0100000US,United States,None,211,<NA>,Oil and gas extraction,6409,229734250,12624930,125129,2017
2,0100000US,United States,None,2111,<NA>,Oil and gas extraction,6409,229734250,12624930,125129,2017
3,0100000US,United States,None,21111,<NA>,Oil and gas extraction,6409,229734250,12624930,125129,2017
4,0100000US,United States,None,211111,<NA>,Crude petroleum and natural gas extraction,5997,197766167,11918513,116968,2017
...,...,...,...,...,...,...,...,...,...,...,...
3829,0100000US,United States,None,813910,<NA>,Business associations,16522,23724292,7380717,112831,2012
3830,0100000US,United States,None,81392,<NA>,Professional organizations,6992,18565199,5879209,84198,2012
3831,0100000US,United States,None,813920,<NA>,Professional organizations,6992,18565199,5879209,84198,2012
3832,0100000US,United States,None,81399,<NA>,"Other similar organizations (except business, ...",20608,22116835,4948577,135074,2012


We don't use the obsolete/limited SIC2 and SIC4 data from US Census